In [1]:
import setup
setup.init_django()

0pEnqPfhRk7bYu5LSa7ppMuKfqr21kck


In [2]:
import helpers.clients as helper_clients

In [3]:
company_name = "NVDA"
company_ticker = "NVDA"
multiplier = 1
from_date = "2025-01-01"
to_date = "2025-01-20"

In [4]:
client = helper_clients.PolygonAPIClient(
    ticker=company_ticker,
    multiplier=multiplier,
    from_date=from_date,
    to_date=to_date
)
dataset = client.get_stock_data()

dataset_c =  client.get_sma_data()
dataset_d = client.get_ema_data()
dataset_e = client.get_macd_data()
dataset_f = client.get_rsi_data()
print(len(dataset))

print(len(dataset_c))
print(len(dataset_d))
print(len(dataset_e))
print(len(dataset_f))

# merge into original data by time index. 
print(dataset[0]['raw_timestamp'])
print(dataset_e[0]['raw_timestamp'])
print(dataset_e[0])


# Merge additional datasets into the original dataset by matching raw_timestamp
for item in dataset:
    # Find matching data in dataset_c, dataset_d, dataset_e, and dataset_f by raw_timestamp
    matching_c = next((c for c in dataset_c if c['raw_timestamp'] == item['raw_timestamp']), None)
    matching_d = next((d for d in dataset_d if d['raw_timestamp'] == item['raw_timestamp']), None)
    matching_e = next((e for e in dataset_e if e['raw_timestamp'] == item['raw_timestamp']), None)
    matching_f = next((f for f in dataset_f if f['raw_timestamp'] == item['raw_timestamp']), None)
    
    # Add the values to the original dataset
    item['sma_value'] = matching_c['value'] if matching_c else None
    item['ema_value'] = matching_d['value'] if matching_d else None
    item['macd_histogram'] = matching_e['histogram'] if matching_e else None
    item['macd_signal'] = matching_e['signal']if matching_e else None 
    item['macd_value'] = matching_e['value'] if matching_e else None
    item['rsi_value'] = matching_f['value'] if matching_f else None

# Print the updated dataset for verification
for item in dataset:
    print(item)
    break  # Remove this break to print all items


    

https://api.polygon.io/v1/indicators/sma/NVDA/?timestamp.gte=2025-01-01&timestamp.lte=2025-01-20&timespan=minute&adjusted=True&window=50&series_type=close&order=desc&limit=5000&apiKey=0pEnqPfhRk7bYu5LSa7ppMuKfqr21kck
5000
5000
5000
5000
5000
1737161940000
1737161940000
{'histogram': 0.00588073703393418, 'signal': 0.0038810830418892, 'value': 0.00976182007582338, 'raw_timestamp': 1737161940000, 'time': datetime.datetime(2025, 1, 18, 0, 59, tzinfo=<UTC>)}
{'open_price': 137.639, 'close_price': 137.631, 'high_price': 137.65, 'low_price': 137.6, 'number_of_trades': 263, 'volume': 15287, 'volume_weighted_average': 137.6218, 'raw_timestamp': 1737161940000, 'time': datetime.datetime(2025, 1, 18, 0, 59, tzinfo=<UTC>), 'sma_value': 137.57963599999997, 'ema_value': 137.59785696926457, 'macd_histogram': 0.00588073703393418, 'macd_signal': 0.0038810830418892, 'macd_value': 0.00976182007582338, 'rsi_value': 63.55034024085424}


In [5]:
dataset_c

[{'value': 137.57963599999997,
  'raw_timestamp': 1737161940000,
  'time': datetime.datetime(2025, 1, 18, 0, 59, tzinfo=<UTC>)},
 {'value': 137.57962399999997,
  'raw_timestamp': 1737161880000,
  'time': datetime.datetime(2025, 1, 18, 0, 58, tzinfo=<UTC>)},
 {'value': 137.58002399999998,
  'raw_timestamp': 1737161820000,
  'time': datetime.datetime(2025, 1, 18, 0, 57, tzinfo=<UTC>)},
 {'value': 137.58142399999997,
  'raw_timestamp': 1737161760000,
  'time': datetime.datetime(2025, 1, 18, 0, 56, tzinfo=<UTC>)},
 {'value': 137.58242199999998,
  'raw_timestamp': 1737161700000,
  'time': datetime.datetime(2025, 1, 18, 0, 55, tzinfo=<UTC>)},
 {'value': 137.58327999999995,
  'raw_timestamp': 1737161640000,
  'time': datetime.datetime(2025, 1, 18, 0, 54, tzinfo=<UTC>)},
 {'value': 137.58389999999997,
  'raw_timestamp': 1737161580000,
  'time': datetime.datetime(2025, 1, 18, 0, 53, tzinfo=<UTC>)},
 {'value': 137.58430999999996,
  'raw_timestamp': 1737161520000,
  'time': datetime.datetime(2025

In [6]:
new_stock_data = dataset[0]
new_stock_data

{'open_price': 137.639,
 'close_price': 137.631,
 'high_price': 137.65,
 'low_price': 137.6,
 'number_of_trades': 263,
 'volume': 15287,
 'volume_weighted_average': 137.6218,
 'raw_timestamp': 1737161940000,
 'time': datetime.datetime(2025, 1, 18, 0, 59, tzinfo=<UTC>),
 'sma_value': 137.57963599999997,
 'ema_value': 137.59785696926457,
 'macd_histogram': 0.00588073703393418,
 'macd_signal': 0.0038810830418892,
 'macd_value': 0.00976182007582338,
 'rsi_value': 63.55034024085424}

In [7]:
from market.models import Company, StockQuote

In [8]:
company_obj, created = Company.objects.get_or_create(name=company_name, ticker=company_ticker)
# company_obj, created = Company.objects.create(name=company_name, ticker=company_ticker)
# company_obj, created = Company.objects.get(name=company_name, ticker=company_ticker)





In [9]:
company_obj

<Company: Company object (12)>

In [10]:
new_quotes = []
for data in dataset:
    new_quotes.append(
        StockQuote(company = company_obj, **data)
    )
StockQuote.objects.bulk_create(new_quotes, ignore_conflicts=True)

[<StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: StockQuote object (None)>,
 <StockQuote: St

In [11]:
# new_quotes = []

# for data in dataset:
#     new_quotes.append(
#         StockQuote(company=company_obj, **data)
#     )



In [15]:
StockQuote.objects.all()

<QuerySet [<StockQuote: StockQuote object (137223)>, <StockQuote: StockQuote object (137224)>, <StockQuote: StockQuote object (137225)>, <StockQuote: StockQuote object (137226)>, <StockQuote: StockQuote object (137227)>, <StockQuote: StockQuote object (137228)>, <StockQuote: StockQuote object (137229)>, <StockQuote: StockQuote object (137230)>, <StockQuote: StockQuote object (137231)>, <StockQuote: StockQuote object (137232)>, <StockQuote: StockQuote object (137233)>, <StockQuote: StockQuote object (137234)>, <StockQuote: StockQuote object (137235)>, <StockQuote: StockQuote object (137236)>, <StockQuote: StockQuote object (137237)>, <StockQuote: StockQuote object (137238)>, <StockQuote: StockQuote object (137239)>, <StockQuote: StockQuote object (137240)>, <StockQuote: StockQuote object (137241)>, <StockQuote: StockQuote object (137242)>, '...(remaining elements truncated)...']>

In [16]:
batch_size = 1000
for i in range(0, len(dataset), batch_size):
    batch_chunk = dataset[i:i+batch_size]
    chunked_quotes = []
    for data in batch_chunk:
        chunked_quotes.append(
            StockQuote(company=company_obj, **data)
        )
    StockQuote.objects.bulk_create(chunked_quotes, ignore_conflicts=True)

In [17]:
StockQuote.objects.all().count()

64375